In [1]:
import os, sys
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import datetime
import time

# df_db의 reverse 방향 노선 생성

In [107]:
df_db = pd.read_csv(f"./final_db/df_db_hday.csv", index_col=[0])
df_db = df_db.reindex(columns=['holiday', 'date', 'time', 'line', 'order', 'from_stt', 'to_stt'])
df_db['order']='상선'


In [108]:
df_rvs = df_db.copy()
df_rvs.columns = ['holiday', 'date', 'time', 'line', 'order', 'from_stt', 'to_stt']
df_rvs = df_rvs.reindex(columns=['holiday', 'date', 'time', 'line', 'order', 'to_stt', 'from_stt'])
df_rvs.columns = ['holiday', 'date', 'time', 'line', 'order', 'from_stt', 'to_stt']
df_rvs['order'] = '하선'

In [109]:
df_db = df_db.append(df_rvs).reset_index(drop=True)
df_db['in'] = np.nan
df_db['out'] = np.nan
df_db['crowd'] = np.nan

In [110]:
df_db = df_db.drop_duplicates()
df_db = df_db.reset_index(drop=True)
df_db.to_csv(f"./final_db/df_db_hday_rvs.csv")

In [112]:
df_db

,holiday,date,time,line,order,from_stt,to_stt,in,out,crowd
0,2,2015-01-01,5,1호선,상선,서울역,시청,NaN,NaN,NaN
1,2,2015-01-01,5,1호선,상선,시청,종각,NaN,NaN,NaN
2,2,2015-01-01,5,1호선,상선,종각,종로3가,NaN,NaN,NaN
3,2,2015-01-01,5,1호선,상선,종로3가,종로5가,NaN,NaN,NaN
4,2,2015-01-01,5,1호선,상선,종로5가,동대문,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1605995,0,2015-12-31,24,4호선,하선,쌍문,수유,NaN,NaN,NaN
1605996,0,2015-12-31,24,4호선,하선,창동,쌍문,NaN,NaN,NaN
1605997,0,2015-12-31,24,4호선,하선,노원,창동,NaN,NaN,NaN
1605998,0,2015-12-31,24,4호선,하선,상계,노원,NaN,NaN,NaN


# df_inout 생성

In [14]:
# inout 데이터 새로 가공
df_inout = pd.read_csv(f"./database/Subway_day_hour/2015_subway_day_hour.csv", encoding='cp949', index_col=[0]).reset_index()
df_inout['역명'] = df_inout['역명'].str.replace(r"\([^)]*\)", "", regex=True)
for col in df_inout.columns[4:]:
    df_inout.loc[: , col] = df_inout.loc[: , col].str.replace(",", "")
df_inout['line'] = df_inout['역번호']//100
df_inout = df_inout.drop(columns = ["역번호"])
#df_inout.loc[df_inout['역명'] == '까치산', 'line'] = 2


df_inout.columns = ['date', 'station', 'inout', '5', '6', '7', '8', '9', '10', '11', '12', 
              '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', 'line']
df_inout = df_inout.reindex(columns = ['date', 'line', 'station', 'inout', '5', '6', '7', '8', '9', '10', '11', '12', 
                           '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',])

# 1, 2, 3, 4호선만 남기기
df_inout = df_inout.loc[df_inout['line'].apply(lambda x: x in [1, 2, 3, 4])]
df_inout = df_inout.sort_values(by=["date", "line"]).reset_index(drop=True)

# 2호선 내선/외선 이외의 분기 노선 모두 삭제 
line2_ex_list = ['까치산', '신정네거리', '양천구청', '도림천', '용답', '신답', '용두', '신설동']
cond = df_inout.loc[:, 'station'].apply(lambda x: x in line2_ex_list)

drop_idx = df_inout.loc[(df_inout['line']==2) & cond, :].index
df_inout = df_inout.drop(index=drop_idx)

df_inout = df_inout.drop_duplicates()
df_inout = df_inout.reset_index(drop=True)

/Users/bowoojang/Desktop/idk/venv/univ_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
# inout 데이터 새로 가공
df_tmp = pd.read_csv(f"./database/Subway_day_hour/2016_subway_day_hour.csv", encoding='cp949', index_col=[0]).reset_index()
df_tmp['역명'] = df_tmp['역명'].str.replace(r"\([^)]*\)", "", regex=True)
for col in df_tmp.columns[4:]:
    df_tmp.loc[: , col] = df_tmp.loc[: , col].str.replace(",", "")
df_tmp['line'] = df_tmp['역번호']//100
df_tmp = df_tmp.drop(columns = ["역번호"])
#df_tmp.loc[df_tmp['역명'] == '까치산', 'line'] = 2


df_tmp.columns = ['date', 'station', 'inout', '5', '6', '7', '8', '9', '10', '11', '12', 
              '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', 'line']
df_tmp = df_tmp.reindex(columns = ['date', 'line', 'station', 'inout', '5', '6', '7', '8', '9', '10', '11', '12', 
                           '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',])

# 1, 2, 3, 4호선만 남기기
df_tmp = df_tmp.loc[df_tmp['line'].apply(lambda x: x in [1, 2, 3, 4])]
df_tmp = df_tmp.sort_values(by=["date", "line"]).reset_index(drop=True)

# 2호선 내선/외선 이외의 분기 노선 모두 삭제 
line2_ex_list = ['까치산', '신정네거리', '양천구청', '도림천', '용답', '신답', '용두', '신설동']
cond = df_tmp.loc[:, 'station'].apply(lambda x: x in line2_ex_list)

drop_idx = df_tmp.loc[(df_tmp['line']==2) & cond, :].index
df_tmp = df_tmp.drop(index=drop_idx)

df_tmp = df_tmp.drop_duplicates()
df_tmp = df_tmp.reset_index(drop=True)

In [18]:
# 전처리: na 값들 해결 -> 2016 년 데이터에는 결측치가 없다! 그러므로 여기서 가져오장! 
# (df_inout.groupby('station').sum() - df_tmp.groupby('station').sum()).mean()
# 결과가 -5.675676이므로 그냥 가져오자. 몰랑........ㅎ
st_t = time.time()
print(f"Start {time.time()-st_t}")

for idx, row in df_inout.loc[df_inout['5'].isna()].iterrows():
    #idx, row['date'], row['line'], row['station'], row['inout'], row['5']
    cond1 = df_tmp['date'].str[-5:] == row['date'][-5:]
    cond2 = (df_tmp['station'] == row['station'])& (df_tmp['line'] == row['line'])
    cond3 = df_tmp['inout'] == row['inout']
    cond = cond1&cond2&cond3
    
    df_inout.loc[idx, '5'] = df_tmp.loc[cond,'5'].values[0]
    
print(f"End {time.time()-st_t}")


Start 4.506111145019531e-05
End 1142.7613661289215


In [19]:
# 전처리! inout에 있는 신천 -> 잠실새내
df_inout['station'] = df_inout['station'].replace("신천", "잠실새내")


In [58]:
df_keep = df_inout.copy()
df_keep = df_keep.drop_duplicates()
df_keep = df_keep.reset_index(drop=True)
df_keep.to_csv("./final_db/2015_subway_day_hour_1234_wo_ch3.csv")

In [89]:
df_inout = pd.read_csv("./final_db/2015_subway_day_hour_1234_wo_ch3.csv", index_col=[0])

In [90]:
# 3호선 충무로 승차, 하차 row append 하기.
df_inout = df_inout.append(pd.DataFrame({'date': [date for date in df_inout['date'].unique()],
                            'line': 3, 'station': '충무로', 'inout': '승차'
                           }))

In [92]:
df_inout = df_inout.append(pd.DataFrame({'date': [date for date in df_inout['date'].unique()],
                            'line': 3, 'station': '충무로', 'inout': '하차'
                           }))

In [95]:
df_inout = df_inout.reset_index(drop=True)

In [102]:
line3_list = ['금호', '약수', '동대입구', '충무로', '을지로3가', '종로3가', '안국']

st_t = time.time()
for date in df_inout['date'].unique():
    cond1 = (df_inout['date']==date)&(df_inout['line']==3)
    cond2 = df_inout['station'].isin(line3_list)
    
    cond3 = df_inout['inout'] == '승차'
    cond4 = df_inout['inout'] == '하차'
    
    cond5 = df_inout['station'] == '충무로'
    
    df_3ch_in = df_inout.loc[cond1&cond2&cond3].set_index('station').reindex(line3_list)
    df_3ch_in = df_3ch_in.interpolate(axis=0)
    df_inout.loc[cond1&cond3&cond5, '5':] = df_3ch_in.loc['충무로', '5':].to_numpy()
    
    df_3ch_out = df_inout.loc[cond1&cond2&cond3].set_index('station').reindex(line3_list)
    df_3ch_out = df_3ch_out.interpolate(axis=0)
    df_inout.loc[cond1&cond4&cond5, '5':] = df_3ch_out.loc['충무로', '5':].to_numpy()
print(time.time()-st_t)

15.955717086791992


In [105]:
df_inout = df_inout.drop_duplicates()
df_inout = df_inout.reset_index(drop=True)
df_inout.to_csv("./final_db/2015_subway_day_hour_1234.csv")

In [106]:
df_inout

,date,line,station,inout,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
0,2015-01-01,1,서울역,승차,441.0,399.0,572.0,1106.0,1474.0,2020.0,...,4036.0,4030.0,3666.0,3738.0,3091.0,2749.0,3247.0,2437.0,1279.0,73.0
1,2015-01-01,1,시청,승차,101.0,95.0,104.0,127.0,233.0,308.0,...,1038.0,1027.0,1176.0,1133.0,850.0,950.0,612.0,497.0,192.0,6.0
2,2015-01-01,1,종각,승차,898.0,482.0,281.0,291.0,328.0,449.0,...,1530.0,2012.0,2176.0,2047.0,1767.0,1834.0,1825.0,1666.0,736.0,37.0
3,2015-01-01,1,종로3가,승차,371.0,212.0,155.0,189.0,347.0,491.0,...,1537.0,1791.0,1954.0,1526.0,1297.0,1149.0,1257.0,953.0,415.0,15.0
4,2015-01-01,1,종로5가,승차,57.0,101.0,83.0,128.0,209.0,323.0,...,915.0,984.0,948.0,962.0,722.0,576.0,431.0,241.0,102.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82485,2015-12-27,3,충무로,하차,26.0,45.5,59.0,91.5,132.5,215.5,...,418.0,912.5,923.0,671.5,428.5,288.0,362.5,211.5,65.0,0.0
82486,2015-12-28,3,충무로,하차,29.0,82.0,161.0,231.0,241.5,251.0,...,567.5,656.5,938.5,2220.5,1429.0,842.0,1127.5,751.0,271.0,41.5
82487,2015-12-29,3,충무로,하차,33.5,83.0,163.0,249.0,227.5,271.0,...,601.5,664.0,1082.0,2208.5,1316.0,775.0,828.5,612.5,313.0,47.5
82488,2015-12-30,3,충무로,하차,35.0,77.5,158.5,250.5,223.5,291.5,...,608.0,759.5,1261.5,2387.5,1378.5,768.5,1158.5,736.5,364.0,74.0
